In [266]:
import re
import numpy
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

#ler um pgm
def read_pgm(filename, byteorder='>'):
    with open(filename, 'rb') as f:
        buffer = f.read()
    try:
        header, width, height, maxval = re.search(
            b"(^P5\s(?:\s*#.*[\r\n])*"
            b"(\d+)\s(?:\s*#.*[\r\n])*"
            b"(\d+)\s(?:\s*#.*[\r\n])*"
            b"(\d+)\s(?:\s*#.*[\r\n]\s)*)", buffer).groups()
    except AttributeError:
        raise ValueError("Not a raw PGM file: '%s'" % filename)
    return numpy.frombuffer(buffer,
                            dtype='u1' if int(maxval) < 256 else byteorder+'u2',
                            count=int(width)*int(height),
                            offset=len(header)
                            ).reshape((int(height), int(width)))

# Plotar um rosto 
def plotar_rosto(rosto):
    rosto = np.resize(rosto, (112, 92))
    plt.imshow(rosto, plt.cm.gray)
    plt.show()

def plotar_rosto_save(rosto, name):
    fig = plt.figure()
    rosto = np.resize(rosto, (112, 92))
    plt.imshow(rosto, plt.cm.gray)
    fig.savefig(name + '.jpg')
    
# PCA
def PCA_algoritmo(treino, n_componentes):
    pca = PCA(n_components=n_componentes) # número de componentes escolhido
    pca.fit(treino)
    componentes = pca.components_ 
    treinocomp = pca.transform(treino) 
    return componentes, treinocomp

In [236]:
# Separação em Treino e Teste 
teste = []
treino = []
total = []
for pessoa in range(40):
    for foto in range(10):
        if foto < 7:
            treino += [np.resize(read_pgm("s{0}/{1}.pgm".format(pessoa+1, foto+1)), 112*92)]
        else:
            teste += [np.resize(read_pgm("s{0}/{1}.pgm".format(pessoa+1, foto+1)), 112*92)]
total = teste + treino


In [237]:
componentes, treinocomp = PCA_algoritmo(treino, 20)

In [238]:
# Obtenção do rosto a partir das componentes
def obterrosto(i, componentes, treinocomp):
    rosto = None
    for j in range(len(componentes)):
        if j == 0:
            rosto = treinocomp[i][j]*componentes[j]
        else:
            rosto = np.sum([rosto, treinocomp[i][j]*componentes[j]], axis=0)
    return rosto

In [241]:
plotar_rosto(obterrosto(10, componentes, treinocomp))
plotar_rosto(treino[10])

In [242]:
def erro_quadratico(x, y):
    erro = 0
    for i in range(len(x)):
        erro += (x[i] - y[i])**2
    return erro

def var_erro(n_componentes):
    componentes, treinocomp = PCA_algoritmo(treino, n_componentes)
    erros = []
    for i in range(len(teste)):
        erros += [erro_quadratico(obterrosto(i, componentes, treinocomp), treino[i])]
    return np.var(erros)

In [256]:
# Plot da variância
x = []
y = []
for i in range(0, 30):
    x += [i+1]
    y += [var_erro(i+1)]
fig = plt.figure()
plt.plot(x, y, '-k')
plt.ylabel('Variância do erro Quadrático')
plt.xlabel('Componentes', fontsize=14)
fig.savefig('test.jpg')
plt.show()

In [267]:
for i in range (5, 40):
    componentes, treinocomp = PCA_algoritmo(treino, i)
    rosto = obterrosto(0, componentes, treinocomp)
    plotar_rosto_save(rosto, 'rosto-' + str(i+1))
plotar_rosto_save(teste[0], 'rosto-correto')

C:\Users\eduar\Anaconda3\lib\site-packages\matplotlib\pyplot.py:516: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
